In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [5]:
## Load ANN model, scalar pkl, one hot encoding pkl
model = load_model('model.h5')

## loading the encoder and scaler
with open('LabelEncoder_gender.pkl', 'rb') as file:
    label_encoder_gen = pickle.load(file)

with open('OneHotEncoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [26]:
# Example input 
input_data = {
    'CreditScore': 619,
    'Geography': 'France',
    'Gender' : 'Female',
    'Age' : 42,
    'Tenure' : 2,
    'Balance' : 0.00,
    'NumOfProducts' : 1,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary': 101348.88
} # in here Geography and Gender are categorical featuers so we need to convert it into numerical features

# 
geo_encoder = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoder_df = pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

c:\Users\albin\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [27]:
# converting it into dataframe
input_data = pd.DataFrame([input_data])

In [28]:
# now converting gender column
input_data['Gender'] = label_encoder_gen.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.0,1,1,1,101348.88


In [29]:
# combing one-hot encoding 
# input_data = pd.concat([input_data.reset_index(drop=True), geo_encoder_df], axis=1)
input_data = pd.concat([input_data.drop('Geography',  axis=1), geo_encoder_df],axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.0,1,1,1,101348.88,1.0,0.0,0.0


In [31]:
# scaling the data
input_scaled = scaler.transform(input_data)
input_scaled

array([[-0.32873352, -1.10191942,  0.29007331, -1.04143297, -1.22605881,
        -0.91025649,  0.64104192,  0.97067965,  0.02822301,  1.00175153,
        -0.57831252, -0.57773517]])

In [32]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step


array([[0.29468462]], dtype=float32)

In [33]:
prediction_prob = prediction[0][0]
prediction_prob


0.29468462

In [34]:
if prediction_prob > 0.5:
    print('The customer is likely to churn.')
else:
    print('Not likely to churn.')

Not likely to churn.
